In [14]:
import torch
import torchvision
from torchvision import transforms, datasets

In [15]:
train = datasets.MNIST("", train=True, download=True,
    transform = transforms.Compose([transforms.ToTensor()]))

test = datasets.MNIST("", train=False, download=True,
    transform = transforms.Compose([transforms.ToTensor()]))

100.1%Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw
113.5%Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw
100.4%Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw
180.4%Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw
Processing...
Done!


In [16]:
trainset = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=10, shuffle=True)